In [1]:
import os
import re
import csv
import sys
import math
import heapq
import shutil
import queue as Q
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
import nltk
from model import TrieNode

In [2]:
def shorten(s):
    superfluous = ' 啊吧哇呀嘛吗耶？！?!.。诶了阿哈呗啥'
    for i in superfluous:
        s = s.rstrip(i).lower()
    ss= ''.join(reversed(s))
    k = -1
    j = 0
    array = [-1]
    plen = len(ss)
    zeros = 0
    nonze = 0
    while j < plen:
        if k == -1 or ss[j] == ss[k]:
            k += 1
            j += 1
            if k == 0:
                if j != zeros + 1:
                    loop = (int)(nonze/zeros) -1
                    return cut_string(s, plen) if loop<1 else cut_string(s, plen - zeros*loop)
                zeros += 1
            else:
                nonze += 1
            array.append(k)
        else:
            k = array[k]
    return cut_string(s, plen) if zeros==plen else (cut_string(s, zeros) if zeros>1 else cut_string(s, zeros+1)) 
def cut_string(s, lens):
    return s[0: lens]

In [3]:
def arrayTodict(lines):
    word_dict_l={}
    for line in lines:
        line=shorten(line.strip('\n').replace(' ', ''))
        if len(line)<2 or bool(re.search(r'\d', line)):
            continue
        if word_dict_l.get(line):
            word_dict_l[line] += 1
        else:
            word_dict_l[line] = 1
    return word_dict_l

In [4]:
def dict_union(d1, d2):
    keys = d1.keys() | d2.keys()
    temp = {}
    for key in keys:
        temp[key] = sum([d.get(key,0) for d in (d1, d2)])
    return temp

In [5]:
def cut1(word_dict):
    #stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
    counts = dict()
    for line in word_dict:
        num = word_dict[line]
        words = jieba.lcut(line)
        if len(words) > 8:
            continue
        for word in words:
            if len(word) == 1:
                continue
            else:
                counts[word] = counts.get(word, 0) + num
    return counts

In [6]:
def build_c(i, terms_id):
    list1 = []
    path = "chunks_fanju/chunk_"+str(i)+".txt"
    list1 = sorted(terms_id.items(), key=lambda item: item[0])
    with open(path, 'w', newline='') as f:
        for key in list1:
            list2= []
            f.write(str(key[0]))
            for word in key[1]:
                f.write(',')
                f.write(str(word[0]))
                f.write('-')
                f.write(str(word[1]))
            f.write('\n')
    f.close()

In [7]:
def jiebaUtil():
    new_words = []
    f = open("new_words_all.txt")
    lines = f.readlines()
    for line in lines:
        split = line.strip().split(',')
        new_words.append((split[0],int(split[1])))
    f.close()
    for key_word in new_words:
        if key_word[1] > 2:
            jieba.add_word(key_word[0], freq = key_word[1]*10, tag = None)

In [8]:
def construct(intToFanju):
    i = 0
    j = -1000
    k = 0
    terms_id = dict()
    doc_len = dict()
    counts = dict()
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) == 0:
            key = j
            for word in counts:
                if word.isalpha() and counts[word]<3:
                    continue
                if key in doc_len:
                    doc_len[key] += counts[word]
                else:
                    doc_len[key] = counts[word]
                if word in terms_id:
                    terms_id[word].append((key,counts[word]))
                else: 
                    terms_id[word] = [(key,counts[word])]
            counts = dict()
            if j>0:
                build_c(j, terms_id)
                terms_id = dict()
            j+=1000
            intToFanju[j] = root
        else:
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                texts_l = arrayTodict(lines)
                counts = dict_union(counts, cut1(texts_l))
    build_c(j, terms_id)
    terms_id = dict()
    return doc_len

In [16]:
def count(intToFanju):
    counts = dict()
    cnt = 0
    for root, dirs, files in os.walk(ori_path, topdown=True):
        if '\.ipynb_checkpoints' in root:
            continue
        if len(files) != 0:
            for name in files:
                f = open(root+"\\"+ name, errors='ignore')
                lines = f.readlines()
                for line in lines:
                    cnt+=1
                    line=shorten(line.strip('\n').replace(' ', ''))
                    c = len(line)
                    if c in counts:
                        counts[c] +=1
                    else:
                        counts[c] = 1
                    if cnt%100000==0:
                        print(cnt)
    #build_c(j, terms_id)
    #terms_id = dict()
    return counts,cnt

In [10]:
intToJuji = dict()
intToFanju= dict()
ori_path = '动漫'
files= os.listdir(ori_path)

In [9]:
jiebaUtil()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.878 seconds.
Prefix dict has been built successfully.


In [14]:
doc_len = construct(intToFanju)

In [21]:
cnt

59771835

In [23]:
counts[1], counts[2],counts[3],counts[4],counts[5],counts[6]

(787620, 7756349, 6872931, 11008786, 7236468, 6035023)

In [17]:
counts,cnt = count(intToFanju)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [28]:
N = len(intToJuji)

In [37]:
fanju_doc = dict()
for i in doc_len:
    a = int(i/1000)
    if a in fanju_doc:
        fanju_doc[a] +=  doc_len[i]
    else:
        fanju_doc[a] =  doc_len[i]

In [38]:
fanju_doc

{1: 13090,
 2: 11002,
 3: 147232,
 4: 46628,
 5: 18507,
 6: 139917,
 7: 138244,
 8: 114659,
 9: 149765,
 10: 163379,
 11: 145467,
 12: 141291,
 13: 135446,
 14: 44461,
 15: 151273,
 16: 60817,
 17: 174818,
 18: 2488,
 19: 97344,
 20: 6044,
 21: 2558,
 22: 44923,
 23: 4603,
 24: 13184,
 25: 14312,
 26: 1541,
 27: 31866,
 28: 235053,
 29: 9432,
 30: 91074,
 31: 109330,
 32: 699,
 33: 240536,
 34: 39330,
 35: 82876,
 36: 13150,
 37: 132551,
 38: 159899,
 39: 212,
 40: 13190,
 41: 126700,
 42: 90280,
 43: 71595,
 44: 3264,
 45: 1191,
 46: 32188,
 47: 363098,
 48: 178570,
 49: 290861,
 50: 181100,
 51: 149980,
 52: 152407,
 53: 16422,
 54: 153095,
 55: 132740,
 56: 37329,
 57: 3392,
 58: 44227,
 59: 237,
 60: 3118,
 61: 814,
 62: 146985,
 64: 206599,
 65: 16350,
 66: 117190,
 67: 134747,
 68: 35558,
 69: 62903,
 70: 39071,
 71: 41050,
 72: 66304,
 73: 237870,
 74: 147939,
 75: 18844,
 76: 15855,
 77: 222330,
 78: 154929,
 79: 9979,
 80: 156138,
 81: 48738,
 82: 154102,
 83: 173855,
 84: 804

In [36]:
def writeJuji(intToJuji):
    f = open('juji.txt','w', encoding="utf-8")
    for juji in intToJuji:
        f.write(str(juji))
        f.write(',')
        f.write(intToJuji[juji])
        f.write('\n')
    f.close()

In [30]:
def build_blocks_heap(N):
    reader = txt_reader('chunks_fanju')
    heap = []
    for i in range(len(reader)):
        next_line = reader[i].readline().strip().split(',')
        heapq.heappush(heap, (next_line[0], next_line[1:], i))

    terms_id = dict()
    j = 0
    next_term = ''
    try:
        while not len(heap) == 0:
            next_index = heapq.heappop(heap)
            judge = (next_index[0] == next_term)
            next_term = next_index[0]
            next_line = reader[next_index[2]].readline()
            if next_line:
                next_line= next_line.strip().split(',')
                heapq.heappush(heap, (next_line[0], next_line[1:], next_index[2]))
            else:
                reader[next_index[2]].close()
            if (sys.getsizeof(terms_id) > 50000000 and not judge):
                j = j + 1
                build_b(j, terms_id, N)
                terms_id = dict()
            if next_term in terms_id:
                terms_id[next_term].extend(next_index[1])
            else: 
                terms_id[next_term] = next_index[1]
    finally:
        j=j+1
        build_b(j, terms_id, N)
        return terms_id

In [31]:
def build_b(i, terms_id, N):
    path = "blocks_fanju/block_"+ str(i) +".csv"
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        try:
            for key in terms_id:
                if len(terms_id[key]) == 1:
                    continue
                list2= []
                list2.append(key)
                idf = math.log(N /(len(terms_id[key])+1), 10)
                list2.append(idf)
                list2.extend(terms_id[key])
                writer.writerow(list2)
        finally:
            f.close()

In [106]:
def readBlock():
    f = open('blocks_new/block_1.csv','r')
    lines = f.readlines()
    doc_len2 = []
    for line in lines:
        split = line.split(',')
        doc_len2.append((split[0],float(split[1]),split[2:]))
    f.close()
    return doc_len2

In [107]:
blocks = readBlock()

In [108]:
blocks[:3]

[('__',
  1.2466457889088292,
  ['10001-17',
   '10003-2',
   '10004-4',
   '10010-4',
   '10015-2',
   '1006010-9',
   '1016001-16',
   '1016003-1',
   '1028005-1',
   '1028008-7',
   '1028010-1',
   '1035001-4',
   '1035010-5',
   '1035012-2',
   '1037003-1',
   '1037008-1',
   '1037009-2',
   '1037010-2',
   '1042005-5',
   '1042006-8',
   '1043008-2',
   '1043009-1',
   '1049018-2',
   '1050002-2',
   '1056004-4',
   '1057006-4',
   '1057007-4',
   '1061009-8',
   '1063003-1',
   '1068019-4',
   '1068042-1',
   '1068063-2',
   '107001-5',
   '107003-3',
   '107009-16',
   '107011-25',
   '107012-4',
   '107014-2',
   '107015-2',
   '107017-3',
   '107018-2',
   '107021-11',
   '107022-3',
   '107023-2',
   '107024-12',
   '107025-3',
   '1074008-2',
   '108005-3',
   '108006-3',
   '108009-14',
   '108012-3',
   '108013-1',
   '1083003-4',
   '1090005-1',
   '109001-8',
   '1094005-4',
   '1100001-1',
   '1100006-4',
   '110005-1',
   '110006-1',
   '110008-3',
   '110009-4',
   '1

In [32]:
terms_id = build_blocks_heap(27278)

In [33]:
terms_id['日之呼吸']

['1148000-173',
 '1205000-7',
 '225000-3',
 '250000-4',
 '327000-3',
 '685000-4',
 '774000-11']

In [43]:
    path = "doclen_fanju.txt"
    with open(path, 'w', newline='') as f:
        for key in fanju_doc:
            f.write(str(key*1000))
            f.write('-')
            f.write(str(fanju_doc[key]))
            f.write('\n')
    f.close()

In [26]:
f = open('doclen_new.txt','r')
lines = f.readlines()
doc_len = dict()
for line in lines:
    split = line.split('-')
    doc_len[int(split[0])] = int(split[1].strip())

In [19]:
def txt_reader(path):
    fileList = os.listdir(path)
    files_reader = dict()
    i = 0
    for file in fileList:
        if(file.split('.')[-1] == "txt"):
            name = path+'/'+file
            files_reader[i] = open(name, "r")
            i = i + 1
    return files_reader

In [10]:
path = 'blocks_new'
fileList = os.listdir(path)
files_reader = dict()
for file in fileList:
    if(file.split('.')[-1] == "csv"):
        name = path+'/'+file
        com = re.compile(r"_(\d+).csv", re.M)
        find = com.findall(name)
        i = int(find[0])
        files_reader[i] = name

In [84]:
f = open('juji.txt','r',encoding='UTF-8')
lines = f.readlines()
intToJuji = dict()
for line in lines:
    split = line.split(',')
    key = int(split[0])
    intToJuji[key] = split[1].strip()

In [13]:
filename_index = intToJuji

In [128]:
import math
def hw3_bm25_11811721(query, doc_len={}, dictionary={}, top = 10, k=1.2, b=0.75):
    scores = dict()
    idf = dict()
    N = len(doc_len)
    avg_doclen = sum(doc_len.values())/N
    for doc in doc_len:
        scores[doc] = 0
    for key in query:
        idf[key] = 1.0 + math.log(N / len(dictionary[key]))
        if key not in dictionary.keys():
            continue
        for item in dictionary[key]:
            split = item.split('-')
            juji = int(int(split[0])/1000)
            cnt = int(split[1])
            scores[juji] += idf[key] * (k + 1) * cnt / (k * (1 - b + b*(doc_len[juji] / avg_doclen))+ cnt)
    scores_list = []
    for key in scores:
        scores_list.append((intToFanju[int(key*1000)],scores[key]))
    scores_list.sort(key = lambda x: x[1], reverse = True)
    return scores_list[:top]

In [132]:
terms_id

{'__': ['10000-29',
  '1006000-9',
  '1016000-17',
  '1019000-1',
  '1028000-9',
  '1035000-11',
  '1037000-6',
  '1042000-13',
  '1043000-3',
  '1049000-2',
  '1050000-2',
  '1051000-1',
  '1056000-4',
  '1057000-8',
  '1061000-8',
  '1063000-1',
  '1068000-7',
  '107000-93',
  '1074000-2',
  '108000-24',
  '1083000-4',
  '109000-8',
  '1090000-1',
  '1094000-4',
  '1096000-1',
  '110000-10',
  '1100000-5',
  '1104000-8',
  '1111000-13',
  '1121000-2',
  '1122000-124',
  '1124000-18',
  '1127000-4',
  '1132000-24',
  '1143000-4',
  '1145000-4',
  '1148000-157',
  '1150000-8',
  '1157000-4',
  '1166000-15',
  '1174000-4',
  '1177000-120',
  '1178000-65',
  '1182000-2',
  '1190000-1',
  '1191000-1',
  '1196000-4',
  '1199000-2',
  '12000-3',
  '1201000-8',
  '1205000-9',
  '1210000-11',
  '1212000-2',
  '129000-2',
  '133000-2',
  '153000-20',
  '159000-1',
  '161000-2',
  '163000-1',
  '164000-2',
  '167000-27',
  '168000-35',
  '17000-7',
  '175000-4',
  '179000-6',
  '185000-4',
  '1

In [130]:
hw3_bm25_11811721(['寄刀片'], fanju_doc, terms_id)

[('动漫\\镇魂街第二季', 4.481275366540487),
 ('动漫\\漆黑的子弹', 4.4754974337067805),
 ('动漫\\Charlotte', 4.473226481662018),
 ('动漫\\魔法少女育成计划', 4.469363883925918),
 ('动漫\\秒速五厘米', 4.463466666083921),
 ('动漫\\覆面系NOISE', 4.46265638802014),
 ('动漫\\乌冬面之国的金色毛球', 4.462434807196287),
 ('动漫\\可塑性记忆', 4.462041556595691),
 ('动漫\\神的记事本', 4.453516553496864),
 ('动漫\\君吻PureRouge', 4.45014198298647)]

In [25]:
doc_len[1000004]

KeyError: 1000004

In [71]:
solve('寄刀片', files_reader)

NameError: name 'solve' is not defined

In [19]:
solve('届不到', files_reader)

['届不到']
动漫\钢壳都市雷吉欧斯\15_传达不到的心意
动漫\我的女神\26_啊啊心跳加速就是大人的味道
动漫\魔法老师涅吉第一季\22_苦中作乐是困难的
动漫\Infini-TForce\07_疯狂的父亲
动漫\心跳回忆ONLYLOVE\26_心跳的选择
动漫\RobiHachi\05_有权能使鬼推磨
动漫\白色相簿上半篇章\04_有感到过超越想像的理解吗反过来的时候倒是很多
动漫\牙狼-炎之刻印-\26_【TV未放送】丝车−HOME−
动漫\异界群敌代码重组\09_我可不会说再见哦
动漫\白色相簿上半篇章\11_心中怀抱的东西并不能随时间流去而淡解只是贴上忘却的标签来掩饰


In [20]:
solve('木叶飞舞之处，火亦生生不息', files_reader)

['木叶', '飞舞', '之', '处', '，', '火', '亦', '生生不息']
动漫\火影忍者疾风传\57_木叶秘传祝言日和恭喜结婚
动漫\犬夜叉\145_那个世界的交界处独特的守门人
动漫\火影忍者疾风传\01_叛逃
动漫\机动战士高达SEEDDestinyHD\28_残存与消逝的生命
动漫\RWBY\11_Jaundice
动漫\博人传 火影忍者新时代\139_恐怖鬼熊圆子
动漫\博人传火影忍者新时代\075_龙地洞的试炼
动漫\博人传火影忍者新时代\139_恐怖鬼熊圆子
动漫\博人传 火影忍者新时代\075_龙地洞的试炼
动漫\超时空要塞Frontier\23_真实的开端


In [131]:
hw3_bm25_11811721(['木大'], doc_len, terms_id)

KeyError: 1

In [21]:
solve('卡Q因', files_reader)

['卡Q因']
动漫\FORTUNEARTERIAL-赤之约定-\08_记忆
动漫\JOJO的奇妙冒险-星尘斗士\20_死神13 其二
动漫\魔偶马戏团\32_黑色流星
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\11_赌徒达比 其二
动漫\JOJO的奇妙冒险-星尘斗士\02_由谁进行制裁
动漫\JOJO的奇妙冒险-星尘斗士\09_黄色节制
动漫\JOJO的奇妙冒险-星尘斗士\08_恶魔
动漫\农林\05_五人的四天农
动漫\JOJO的奇妙冒险-不灭钻石\19_胖重的收成者Harvest 其二
动漫\JOJO的奇妙冒险-不灭钻石\32_7月15日四 其二


In [83]:
hw3_bm25_11811721(['黄金精神'], doc_len, terms_id, top = 10)

[(73040, '动漫\\JOJO的奇妙冒险黄金之风\\40_王中之王KingofKings', 11.809690877216028),
 (73011, '动漫\\JOJO的奇妙冒险黄金之风\\11_纳兰迦的航空史密斯', 11.716725036950598),
 (73017, '动漫\\JOJO的奇妙冒险黄金之风\\17_「壮烈成仁」其二', 11.675318724970612),
 (73001, '动漫\\JOJO的奇妙冒险黄金之风\\01_黄金体验GoldExperience)', 11.618150160788932),
 (73030, '动漫\\JOJO的奇妙冒险黄金之风\\30_在那看似即将坠落的天空下', 11.603485596936316),
 (73004, '动漫\\JOJO的奇妙冒险黄金之风\\04_黑帮入门', 11.485274080858314),
 (73018, '动漫\\JOJO的奇妙冒险黄金之风\\18_娃娃脸', 11.387275419495143),
 (49019, '动漫\\Fatestaynight06版\\19_黄金之王', 11.077862522246365),
 (73002, '动漫\\JOJO的奇妙冒险黄金之风\\02_布加拉提到来', 11.032832760529875),
 (73015, '动漫\\JOJO的奇妙冒险黄金之风\\15_前往佛罗伦萨的超特快列车', 11.024742711890237)]

In [74]:
hw3_bm25_11811721(['火之意志'], doc_len, terms_id)

[(292109, '动漫\\博人传火影忍者新时代\\109_汤烟忍法帖·薯片与大岩石', 12.53655522582462),
 (292140, '动漫\\博人传火影忍者新时代\\140_输给薯片的心转身之术', 12.523349513901167),
 (292137, '动漫\\博人传火影忍者新时代\\137_武士留学生', 10.877878024441223),
 (774013, '动漫\\炎炎消防队\\13_设好的陷阱', 10.843418599817932),
 (421007, '动漫\\学战都市Asterisk第二季\\07_战律', 10.8016880652986),
 (292071, '动漫\\博人传火影忍者新时代\\071_全世界最坚硬的石头', 10.60973565549208),
 (292149, '动漫\\博人传火影忍者新时代\\149_朋友', 10.564485862659772),
 (292111, '动漫\\博人传火影忍者新时代\\111_汤烟忍法帖·未来之玉', 10.068819288249278),
 (292110, '动漫\\博人传火影忍者新时代\\110_汤烟忍法帖·复苏温泉', 9.755626144804145),
 (292153, '动漫\\博人传火影忍者新时代\\153_黄金和声', 9.702984211344303)]

In [22]:
solve('黄金精神', files_reader)

['黄金精神']
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\20_亚空瘴气 瓦尼拉·艾斯 其三
动漫\JOJO的奇妙冒险-黄金之风\30_在那看似即将坠落的天空下
动漫\JOJO的奇妙冒险-不灭钻石\39_再会了 杜王町－黄金精神
动漫\JOJO的奇妙冒险-黄金之风\40_王中之王（King of Kings）
动漫\JOJO的奇妙冒险\10_纽约的JoJo
动漫\JOJO的奇妙冒险\23_随风而逝的战士
动漫\JOJO的奇妙冒险-星尘斗士\01_被恶灵附身的男人
动漫\JOJO的奇妙冒险-不灭钻石\18_胖重的收成者Harvest 其一
动漫\JOJO的奇妙冒险黄金之风\30_在那看似即将坠落的天空下
动漫\JOJO的奇妙冒险-不灭钻石\13_捡到麻烦的东西了


In [23]:
solve('高洁的教皇', files_reader)

['高洁', '的', '教皇']
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\22_迪奥的世界 其二
动漫\JOJO的奇妙冒险-星尘斗士\11_皇帝与倒吊人 其二
动漫\JOJO的奇妙冒险-星尘斗士\24_女教皇 其二
动漫\JOJO的奇妙冒险-星尘斗士\04_灰色之塔
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\16_电玩高手达比 其一
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\21_迪奥的世界 其一
动漫\JOJO的奇妙冒险-星尘斗士\02_由谁进行制裁
动漫\JOJO的奇妙冒险-星尘斗士\20_死神13 其二
动漫\盾之勇者成名录\19_四圣勇者
动漫\某科学的超电磁炮T\10_才人工房


In [24]:
solve('典明粥', files_reader)

['典明粥']
动漫\JOJO的奇妙冒险-不灭钻石\13_捡到麻烦的东西了
动漫\JOJO的奇妙冒险-星尘斗士\19_死神13 其一
动漫\JOJO的奇妙冒险-星尘斗士\20_死神13 其二
动漫\漫画家与助手\03_黄色书籍的大慌乱
动漫\品酒要在成为夫妻后\04_第四杯气泡白葡萄酒
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\09_「赛特神」阿雷西 其二
动漫\小鸠\04_绿叶的鼓动
动漫\钢之炼金术师\24_记忆的黏著
动漫\我们仍未知道那天所看见的花的名字\03_寻找面码委员会
动漫\OneRoom第二季\03_花坂结衣被骂了


In [25]:
solve('火之意志', files_reader)

['火之意志']
动漫\魔法老师涅吉第一季\22_苦中作乐是困难的
动漫\博人传 火影忍者新时代\109_汤烟忍法帖·薯片与大岩石
动漫\博人传 火影忍者新时代\140_输给薯片的心转身之术
动漫\神奇宝贝起源\01_Red
动漫\星空清理者\17_所以才是他
动漫\博人传火影忍者新时代\109_汤烟忍法帖·薯片与大岩石
动漫\博人传火影忍者新时代\140_输给薯片的心转身之术
动漫\圣斗士星矢THELOSTCANVAS冥王神话\17_尘芥
动漫\Infini-TForce\10_点燃火焰
动漫\博人传 火影忍者新时代\149_朋友


In [26]:
solve('rerorero', files_reader)

['rero', 'rero']
动漫\JOJO的奇妙冒险-不灭钻石\32_7月15日四 其二
动漫\幽游白书\100_真相大白的邪眼秘密
动漫\JOJO的奇妙冒险-星尘斗士\09_黄色节制
动漫\EVA新世纪福音战士\15_谎言与沉默
动漫\地狱少女三鼎\01_01
动漫\JOJO的奇妙冒险\11_游戏高手
动漫\JOJO的奇妙冒险-星尘斗士\02_由谁进行制裁
动漫\Ｄｒ．ＳＴＯＮＥ 石纪元\13_假面战士
动漫\JOJO的奇妙冒险-星尘斗士\07_力量
动漫\家庭教师HITMANREBORN\070_日常篇入江正一的灾难


In [27]:
solve('但是我拒绝', files_reader)

['但是', '我拒绝']
动漫\不可思议星球的双胞胎公主\49_大家的心意感受到了吗布莱德
动漫\绝对防卫利维坦\12_绝对会受打击的嘛
动漫\OneRoom第二季\08_七桥御乃梨蛮不讲理
动漫\OneRoom\10_青岛萌香陷入烦恼
动漫\青出于蓝\20_治愈
动漫\小绿和小蓝\25_周末篇回忆下
动漫\复仇者世上最强英雄组合\28_S02E02
动漫\Infini-TForce\07_疯狂的父亲
动漫\OneRoom\07_桃原奈月害羞搪塞
动漫\OneRoom\04_花坂结衣牵起手


In [28]:
solve('菠萝头', files_reader)

['菠萝头']
动漫\博人传 火影忍者新时代\143_盯上黑里的犯人
动漫\博人传 火影忍者新时代\113_队长的资质
动漫\博人传 火影忍者新时代\061_砂铁使新希
动漫\博人传 火影忍者新时代\116_木叶丸与柠檬
动漫\博人传 火影忍者新时代\111_汤烟忍法帖·未来之玉
动漫\博人传 火影忍者新时代\124_决断之时
动漫\博人传火影忍者新时代\143_盯上黑里的犯人
动漫\博人传 火影忍者新时代\151_博人与天斗
动漫\博人传 火影忍者新时代\118_吞噬记忆之物
动漫\博人传 火影忍者新时代\128_浦式的目的


In [39]:
hw3_bm25_11811721(['菠萝头'], doc_len, terms_id)

[('动漫\\博人传火影忍者新时代\\116_木叶丸与柠檬', 11.537956799967679),
 ('动漫\\博人传火影忍者新时代\\028_宣战布告', 11.531067802474404),
 ('动漫\\博人传火影忍者新时代\\154_向日葵忍者体验', 11.497539734757005),
 ('动漫\\博人传火影忍者新时代\\143_盯上黑里的犯人', 11.360903248318381),
 ('动漫\\博人传火影忍者新时代\\065_父与子', 11.354928389363973),
 ('动漫\\博人传火影忍者新时代\\151_博人与天斗', 11.312542319645997),
 ('动漫\\博人传火影忍者新时代\\001_漩涡博人', 11.288453475035217),
 ('动漫\\博人传火影忍者新时代\\012_博人和巳月', 11.257427564175085),
 ('动漫\\博人传火影忍者新时代\\113_队长的资质', 11.23995931643851),
 ('动漫\\博人传火影忍者新时代\\153_黄金和声', 11.21412885863027)]

In [29]:
solve('啥是gay', files_reader)

['啥', '是gay']
动漫\小绿和小蓝\59_校园篇之相当现实的一话
动漫\青春草莓蛋\07_憧憬色的蜜粉调理
动漫\家有穆珂\15_铲雪车运来了忧郁
动漫\家有穆珂\20_小松先生牵牛花开了哟
动漫\家有穆珂\11_牛岛拜托看家
动漫\家有穆珂\06_夏天来了台风烟花降温贴
动漫\家有穆珂\14_元旦日出
动漫\家有穆珂\24_环君登场牛岛流泪的原因
动漫\双恋\07_仲夏的宝石箱
动漫\出租魔法使\14_星之记忆


In [30]:
solve('为什么你会这么熟练啊', files_reader)

['为什么', '你会', '这么熟练', '啊']
动漫\白色相簿上半篇章\08_时间不充裕时往往可以做得更多放手去做吧当然恋爱也一样
动漫\白色相簿2\11_蓦然雪化 且静候 飞雪再临之时（后篇）
动漫\CANDYBOY\09_幸福的共有理论
动漫\白色相簿上半篇章\09_令人眷恋的回忆不需要编排剧本从说出口的那一瞬间就注定是戏言
动漫\白色相簿2\11_蓦然雪化且静候飞雪再临之时后篇
动漫\君吻PureRouge\24_24
动漫\白色相簿2\12_毕业
动漫\超时空要塞Macross\14_古力华的报告
动漫\玛纳利亚的密友MysteriaFriends巴哈姆特之怒玛娜利亚魔法学院\09_某一天
动漫\影鳄\10_影鳄


In [31]:
solve('白学现场', files_reader)

['白学现场']
动漫\超时空要塞Macross\29_孤独的歌
动漫\白色相簿上半篇章\08_时间不充裕时往往可以做得更多放手去做吧当然恋爱也一样
动漫\君吻PureRouge\15_15
动漫\我的女神\12_啊啊女神与女王孰轻孰重
动漫\电器街的漫画店\05_春天的裙底风光IamRain喧嚣
动漫\小鸠\18_冬季风的温暖
动漫\双恋\10_三角铃·圣诞
动漫\君吻PureRouge\12_12
动漫\星空清理者\15_她的场合
动漫\MYSELF;YOURSELF\07_老师的牛奶糖


In [64]:
hw3_bm25_11811721(['白学现场'], doc_len, terms_id)

[(1004029, '动漫\\超时空要塞Macross\\29_孤独的歌', 9.980071661740729),
 (1003019, '动漫\\超时空要塞Frontier\\19_三角恋爱', 9.883951600139403),
 (1004036, '动漫\\超时空要塞Macross\\36_温柔的再见', 9.864420180364048),
 (1004034, '动漫\\超时空要塞Macross\\34_私人时间', 9.859441713080797),
 (1004035, '动漫\\超时空要塞Macross\\35_感情', 9.747560075838502),
 (557012, '动漫\\戒律的复活\\12_爱的所在', 9.705934397833946),
 (1006020, '动漫\\超时空要塞Δ\\20_冲动实验', 9.704541230262123),
 (1006012, '动漫\\超时空要塞Δ\\12_风王', 9.648728515127432),
 (1003020, '动漫\\超时空要塞Frontier\\20_钻石裂痕', 9.641587135649715),
 (796013, '动漫\\犬神同学和猫山同学\\13_猫山同学和温泉旅行', 9.641454208915917)]

In [67]:
for i in terms_id['白学现场']:
    s = i.split('-')[0]
    print(i,doc_len[int(s)])

1000006-7 10870
1000012-3 8588
1001004-4 10640
1003002-20 7955
1003003-4 9234
1003004-3 7952
1003005-48 10166
1003006-4 9346
1003010-34 9942
1003011-4 9102
1003015-27 7502
1003016-4 6805
1003019-76 9740
1003020-44 9041
1003025-4 8541
1004017-4 3913
1004021-3 5172
1004023-12 3045
1004028-4 4142
1004029-36 3013
1004032-21 4826
1004033-4 4078
1004034-42 5373
1004035-33 5333
1004036-40 5010
1006005-3 7599
1006006-5 10191
1006009-20 9852
1006012-44 8928
1006013-25 8391
1006014-8 9612
1006016-48 10601
1006017-12 10748
1006019-3 8078
1006020-45 8252
1006021-3 8123
1006022-10 9539
1006023-27 8781
1006024-4 10260
1006026-24 11258
1018011-6 2496
102006-4 10677
102017-12 10840
1028009-3 9327
104012-4 3625
1056002-4 10430
1057005-5 10538
1057009-8 9940
1061006-6 8699
1061010-4 10242
1061012-4 9524
1061013-4 10297
1061020-4 12864
1061022-20 10030
1076004-4 10264
1076005-4 13363
1076012-10 11733
1076021-4 8317
1076022-4 9144
1083002-4 8629
1083005-4 10397
109021-4 10668
1091001-4 3733
11005-20 8574


In [125]:
hw3_bm25_11811721(['大雁'], doc_len, terms_id)

[(292103, '动漫\\博人传火影忍者新时代\\103_迁徙的季节', 16.976498642257624),
 (695010, '动漫\\梅露可物语\\10_无法飞翔的天使和万祈的圣翼前篇', 13.7511812594828),
 (292100, '动漫\\博人传火影忍者新时代\\100_既定的道路', 10.28587177266382),
 (292101, '动漫\\博人传火影忍者新时代\\101_重吾的援军', 10.03852175370824),
 (1143012, '动漫\\高达创形者\\12_光之翼', 9.6526034781786),
 (693003, '动漫\\格林笔记TheAnimation\\03_追忆的灰姑娘', 9.358806080344287),
 (292099, '动漫\\博人传火影忍者新时代\\099_重吾与咒印', 8.86115131714119),
 (1001, '动漫\\0号宿舍\\01_0号宿舍', 0),
 (1002, '动漫\\0号宿舍\\02_怪奇室友', 0),
 (1003, '动漫\\0号宿舍\\03_当世界充满恨', 0)]

In [136]:
hw3_bm25_11811721(['打死','白学家'], doc_len, terms_id)

[(1056010, '动漫\\邻家索菲\\10_吸血鬼与年末', 20.485154210939122),
 (73019, '动漫\\JOJO的奇妙冒险黄金之风\\19_前往威尼斯', 20.243548118514646),
 (1128008, '动漫\\面包带来和平\\08_新开的面包店', 20.11334390390753),
 (154010, '动漫\\三者三叶\\10_鸡肉和蛋糕吃个痛快的日子', 19.48677489276222),
 (678022, '动漫\\来自风平浪静的明天\\22_有所失', 19.164336323276476),
 (850007, '动漫\\真实之泪\\07_说清楚写在这里', 18.958807289571613),
 (698009, '动漫\\梦幻之星online2\\09_战力外通告', 18.377366850125696),
 (73020, '动漫\\JOJO的奇妙冒险黄金之风\\20_白色相簿', 18.345068124545925),
 (866008, '动漫\\神推偶像登上武道馆我就死而无憾\\08_希望能在我的未来中', 18.247237217049104),
 (1035013, '动漫\\轻音少女第一季\\13_冬之日', 17.656928813024457)]

In [124]:
hw3_bm25_11811721(['水调歌头'], doc_len, terms_id)

[(1148004, '动漫\\鬼灭之刃\\04_最终选拔', 14.68233308742768),
 (1148009, '动漫\\鬼灭之刃\\09_手球鬼与箭头鬼', 13.990258018206656),
 (38013, '动漫\\COPCRAFT\\13_TWOWORLDSTWOJUSTICES', 13.849172660143196),
 (770010, '动漫\\灰与幻想的格林姆迦尔\\10_虽然没有领导气质', 13.17406455294443),
 (1148016, '动漫\\鬼灭之刃\\16_让自己以外的某人前进', 12.606266264165958),
 (1148010, '动漫\\鬼灭之刃\\10_一直在一起', 11.805981020490064),
 (371013, '动漫\\多罗罗\\13_白面不动之卷', 11.766711375629669),
 (1000003, '动漫\\超人高中生们即便在异世界也能从容生存\\03_看来露自己决定了自己的价值', 11.707667552617375),
 (463010, '动漫\\川柳少女\\10_七七子和萤火虫与试胆大会', 11.507771213196676),
 (1148013, '动漫\\鬼灭之刃\\13_比生命更重要的东西', 11.460537308465621)]

In [32]:
solve('胃疼', files_reader)

['胃疼']
动漫\双恋\10_三角铃·圣诞
动漫\白色相簿上半篇章\08_时间不充裕时往往可以做得更多放手去做吧当然恋爱也一样
动漫\君吻PureRouge\09_09
动漫\白色相簿上半篇章\10_独角戏之所以空虚并非因为只有一个演员而是观众的目光刺得好痛
动漫\白色相簿上半篇章\06_忘却烦恼的办法对别人的烦恼插上一手如何
动漫\君吻PureRouge\13_13
动漫\白色相簿上半篇章\03_手牵手肩并肩背靠背还有哪怕隔着衣服也好
动漫\白色相簿上半篇章\04_有感到过超越想像的理解吗反过来的时候倒是很多
动漫\白色相簿上半篇章\09_令人眷恋的回忆不需要编排剧本从说出口的那一瞬间就注定是戏言
动漫\白色相簿上半篇章\05_挡在前的不一定是身边的人可越是陌生的人越难以应付


In [33]:
solve('斯特拉斯堡见', files_reader)

['斯特拉斯堡', '见']
动漫\君吻PureRouge\06_06
动漫\白色相簿2\03_轻音乐同好会再结成
动漫\真实之泪\08_没下雪的小镇
动漫\白色相簿2\13_传达不到的爱恋
动漫\宇宙战舰提拉米斯\06_圣夜之乱晦涩的记忆
动漫\斯特拉的魔法\10_精密机械
动漫\恋爱小行星\10_真正的心意
动漫\白色相簿2\04_命运之声
动漫\真实之泪\03_怎么样了之前说的那件事
动漫\无畏魔女\06_召唤好运


In [34]:
solve('世界名画', files_reader)

['世界名画']
动漫\异界群敌代码重组\09_我可不会说再见哦
动漫\异界群敌代码重组\10_异存在伊奈尔玛与神域
动漫\君吻PureRouge\15_15
动漫\异度侵入\13_CHANNELEDⅡ
动漫\逆转裁判第二季\19_华丽的逆转3rdTrial
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\22_迪奥的世界 其二
动漫\星空清理者\07_地球外少女
动漫\星空清理者\15_她的场合
动漫\钢之炼金术师\35_愚者的再会
动漫\EVA新世纪福音战士\24_最后的死使者


In [35]:
solve('打死白学家', files_reader)

['打死白学', '家']
动漫\双恋\08_白色的写生簿
动漫\特别的她\04_Furikiri
动漫\双恋\10_三角铃·圣诞
动漫\一个人的〇〇小日子\10_第一次被说的
动漫\双恋\05_转动的时间
动漫\玉响～hitotose～\12_圣诞夜的故事♪所以
动漫\超时空要塞Macross\15_唐人街
动漫\黄金拼图\11_11
动漫\青出于蓝\14_管饭人
动漫\JOJO的奇妙冒险-黄金之风\20_白色相簿


In [36]:
solve('java', files_reader)

['java']
动漫\历物语\04_历水
动漫\轻音少女第一季\14_LiveHouse
动漫\热血少年\05_不明魔人物体
动漫\粗点心战争2\09_因特罔和超级弹力球
动漫\简单易懂的现代魔法\02_helloworld
动漫\天体的秩序\12_圆盘消失之街
动漫\恋爱小行星\04_激动人心的夏季集训
动漫\兽娘动物园\03_高山
动漫\骑士魔法\01_RobotsFantasy
动漫\此花亭奇谭\03_渴望恋情


In [37]:
solve('python', files_reader)

['py', 'th', 'on']
动漫\小绿和小蓝\04_机器人篇1诞生上
动漫\啾咪BABY日语版\03_超能力诅咒甜蜜相扑
动漫\小绿和小蓝\18_机器人篇4可贵的情感
动漫\会长是女仆大人\06_男鲇泽塾
动漫\小绿和小蓝\71_机器人篇
动漫\Caligula卡里古拉\11_人们总是会寻求正确答案然而正确答案真的能让自己接受吗
动漫\小林家的龙女仆\05_托尔的社会学习虽然她自以为没问题)
动漫\拽妹黛薇儿第一季\02_第二集
动漫\粗点心战争2\09_因特罔和超级弹力球
动漫\小林家的龙女仆\01_史上最强女仆托尔毕竟是龙嘛)


In [38]:
solve('马猴烧酒', files_reader)

['马猴烧酒']
动漫\骇客娃娃\12_魔法少女LovelyHurt❤
动漫\群居姐妹\05_五号楼咲月的动画时钟
动漫\群居姐妹\08_八号楼是姐妹啊
动漫\最游记\38_无法实现的诺言Fleetingvision
动漫\玉响～hitotose～\11_明天的我是怎样的我所以
动漫\浪客剑心\90_风水的奇袭被困的五芒星之谜
动漫\不可思议星球的双胞胎公主\49_大家的心意感受到了吗布莱德
动漫\凸变英雄LEAF\02_东南西北皆“兄弟”
动漫\Re创造主\22_Re创造主
动漫\异界群敌代码重组\12_代码重组


In [39]:
solve('前方高能', files_reader)

['前方高能']
动漫\异度侵入\14_《异度侵入》监督们给中国观众的问候
动漫\斩兽之刃\11_【副导演出道作】附第四集先行预告
动漫\刺客伍六七\03_六七VS十三
动漫\异度侵入ID\14_《异度侵入》监督们给中国观众的问候
动漫\小绿和小蓝\59_校园篇之相当现实的一话
动漫\小绿和小蓝\04_机器人篇1诞生上
动漫\多罗罗\34_注入生命百鬼丸x铃木扩树
动漫\刺客伍六七\15_大保JVS太保丁
动漫\小绿和小蓝\53_网络篇才能下
动漫\小绿和小蓝\10_魔王篇3)魔王城下)


In [40]:
solve('橘里橘气', files_reader)

['橘里橘气']
动漫\小绿和小蓝\52_校园篇之愚人节傲娇
动漫\小绿和小蓝\44_日常篇只说真话的世界
动漫\魔法老师涅吉第一季\21_希望不灭
动漫\钢壳都市雷吉欧斯\09_天剑授受者的资格
动漫\小鸠\09_夏天的记忆
动漫\异界群敌代码重组\09_我可不会说再见哦
动漫\异界群敌代码重组\10_异存在伊奈尔玛与神域
动漫\钢壳都市雷吉欧斯\19_被引导的相遇
动漫\地狱少女三鼎\13_13
动漫\钢壳都市雷吉欧斯\16_洁尔妮失控污染兽袭击


In [41]:
solve('医学奇迹', files_reader)

['医学奇迹']
动漫\异度侵入\13_CHANNELEDⅡ
动漫\星掠者\22_约定
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\24_遥远的旅途 再见了朋友
动漫\CAROLE & TUESDAY\25_AChangeisGonnaCome
动漫\幽游白书\084_藏马的愤怒真面目是谁
动漫\漫画家与助手\08_主编的审查
动漫\卫宫家今天的饭\10_冷了也好吃的炸鸡块
动漫\宝石商人理查德的谜鉴定\01_粉色蓝宝石的正义
动漫\JOJO的奇妙冒险\04_波纹疾走
动漫\刀剑神域Alicization\25_我的英雄


In [44]:
solve('wryyyyyy', files_reader)

['wryy']
动漫\JOJO的奇妙冒险\03_与迪奥度过的青春
动漫\JOJO的奇妙冒险-黄金之风\34_「青春岁月」（Green Day）和「绿洲」（Oasis） 其二
动漫\JOJO的奇妙冒险-黄金之风\19_前往威尼斯
动漫\JOJO的奇妙冒险-星尘斗士-埃及篇\24_遥远的旅途 再见了朋友
动漫\Ｄｒ．ＳＴＯＮＥ 石纪元\09_手握科学的明灯
动漫\JOJO的奇妙冒险-不灭钻石\05_虹村兄弟 其三
动漫\特别的她\04_Furikiri
动漫\JOJO的奇妙冒险\05_黑暗的骑士们
动漫\JOJO的奇妙冒险\08_血战！JoJo&迪奥
动漫\辉夜大小姐想让我告白\08_辉夜大小姐想被称呼 白银御行不能输 于是石上优闭上了双眼


In [45]:
solve('求BGM', files_reader)

['求BGM']
动漫\少年歌行\30_无边落木萧萧下你是我的Superstar
动漫\小绿和小蓝\11_校园篇梦中的你
动漫\心跳回忆ONLYLOVE\26_心跳的选择
动漫\简单易懂的现代魔法\11_quick-and-dirty
动漫\小绿和小蓝\17_网络篇之便利
动漫\神装少女小缠\14_总集篇
动漫\她与她的猫\03_她与她的目光
动漫\幽游白书\090_继承朋友的遗志
动漫\小绿和小蓝\06_魔王篇2)魔王城上)
动漫\战斗女子高校\11_真实


In [46]:
solve('诚哥', files_reader)

['诚哥']
动漫\双恋\09_夕阳的彩色蜡笔
动漫\青春草莓蛋\10_困扰的爱情调色盘
动漫\11eyes\11_名为灭亡的选择
动漫\历物语\12_历死
动漫\地上最强新娘\09_悲恋的女英雄
动漫\青出于蓝\09_一夜
动漫\历物语\02_历花
动漫\黑塚\12_黑乃冢
动漫\心之彼端2\01_新制服
动漫\心之彼端2\13_春风


In [47]:
solve('日之呼吸', files_reader)

['日之呼吸']
动漫\野良神\06_可怕的人
动漫\舞动青春\05_舞伴
动漫\冷然之天秤\11_白壁鸟笼-阴影-
动漫\某科学的一方通行\09_10031次死亡MemoryofDeath
动漫\黑色四叶草\036_三魔眼
动漫\入间同学入魔了\10_激斗处刑玉炮
动漫\命运-冠位指定绝对魔兽战线巴比伦尼亚\08_声东击西作战
动漫\炎炎消防队\12_浅草开战前夜
动漫\刀剑神域 爱丽丝篇 异界战争\10_剑与拳
动漫\炎炎消防队\09_扩燃的恶意


In [48]:
solve('水之呼吸', files_reader)

['水之呼吸']
动漫\品酒要在成为夫妻后\12_第十二杯FirstTime
动漫\Ｄｒ．ＳＴＯＮＥ 石纪元\14_火焰大师
动漫\猫和老鼠旧版\44_老鼠出售MouseforSale
动漫\少年歌行\09_江边论道
动漫\黑色四叶草\034_光魔法VS暗魔法
动漫\来自风平浪静的明天\03_大海的传说
动漫\来自风平浪静的明天\17_两人抱恙
动漫\多罗罗\10_多宝丸之卷
动漫\黑色四叶草\043_神殿大逃杀
动漫\强风吹拂\22_抱紧寂寞


In [49]:
solve('亚撒西', files_reader)

['亚撒西']
动漫\无法逃离的背叛\16_冻结的时间
动漫\OneRoom\11_青岛萌香心中明白
动漫\钢壳都市雷吉欧斯\12_温柔的谎言
动漫\无法逃离的背叛\09_伤痕
动漫\双恋\04_草莓甜饼处等你
动漫\美妙射击部\09_赛场是筒贺
动漫\EVA新世纪福音战士\18_生命的选择
动漫\双恋\13_不远的春天
动漫\徒然喜欢你\02_春天
动漫\青出于蓝\23_决意


In [50]:
solve('电磁炮', files_reader)

['电磁炮']
动漫\翠星之加尔刚蒂亚\10_野心之岛
动漫\浪客剑心\94_风与水的挽歌·如今唯有死力一战
动漫\魔法老师涅吉第一季\08_善恶到头终有报
动漫\精灵宝可梦无印\234_红色暴鲤龙之愤怒
动漫\快盗天使TWINANGEL\09_强敌出现双胞胎幽灵是谁
动漫\战姬绝唱SYMPHOGEAR\12_SYMPHOGEAR
动漫\精灵宝可梦无印\197_再见了菊草叶电气的迷宫
动漫\快盗天使TWINANGEL\10_一气呵成黑暗中燃烧的一朵蔷薇
动漫\爱丽丝学园\09_最喜欢☆萤大人
动漫\CDE\10_人见的世界


In [51]:
solve('霹雳一闪', files_reader)

['霹雳一闪']
动漫\炎炎消防队\05_开战
动漫\Re从零开始的异世界生活\22_怠惰一闪
动漫\我，不是说了能力要平均值么！\11_说了不能松懈的吧
动漫\落第骑士英雄谭\10_深海魔女VS雷切
动漫\刀剑神域 爱丽丝篇 异界战争\08_失格者的烙印
动漫\神薙\04_姐妹
动漫\好想告诉你第一季\10_共同努力
动漫\浪客剑心\54_飞天对缩地宗次郎天赋的能力
动漫\我不是说了能力要平均值么\11_说了不能松懈的吧
动漫\炎炎消防队\20_荣耀加身


In [89]:
text_len = []
for i in doc_len:
    text_len.append((i,doc_len[i],intToJuji[i][3:]))

In [90]:
text_len[:5]

[(1001, 549, '0号宿舍\\01_0号宿舍'),
 (1002, 1241, '0号宿舍\\02_怪奇室友'),
 (1003, 543, '0号宿舍\\03_当世界充满恨'),
 (1004, 1542, '0号宿舍\\04_消失的爱人'),
 (1005, 500, '0号宿舍\\05_作弊疑云')]

In [54]:
len(doc_len)

27314

In [109]:
def insertManyRow(con, strings):
    try:
        cur2 = con.cursor()
        sql2 = "INSERT INTO postgres.ir.reversed_index(term, idf, indices) VALUES(%s,%s,%s)"
        cur2.executemany(sql2, strings)
        con.commit()
    except Exception as e:
        print("执行sql时出错：%s" % (e))
        con.rollback()

In [79]:
import psycopg2
import pandas as pd

connection = psycopg2.connect(user = "checker",
                                  password = "wangxiaohan",
                                  host = "121.36.22.150",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()

In [81]:
connection = psycopg2.connect(user = "checker",
                                  password = "wangxiaohan",
                                  host = "localhost",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()

In [110]:
insertManyRow(connection, blocks)

In [92]:
text_len[:5]

[(1001, 549, '0号宿舍\\01_0号宿舍'),
 (1002, 1241, '0号宿舍\\02_怪奇室友'),
 (1003, 543, '0号宿舍\\03_当世界充满恨'),
 (1004, 1542, '0号宿舍\\04_消失的爱人'),
 (1005, 500, '0号宿舍\\05_作弊疑云')]

In [96]:
len(terms_id)

551286